
# Transport en commun - Données GTFS
## Qu’est-ce que le GTFS ?

**GTFS** (*General Transit Feed Specification*) est un format ouvert pour décrire une **offre de transport en commun** (statique) : lignes, trajets, arrêts, horaires.
Il existe deux familles :

* **GTFS Statique** (ce que nous utilisons ici) : planning de référence, valable sur une période donnée (ex. du 06/06/2025 au 05/08/2025).
* **GTFS-Realtime** : informations en temps réel (retards, positions des véhicules, alertes), sur d’autres flux protobuf. *(non traité ici).*

Pour plus de détails sur ces types **GTFS** voir [https://en.wikipedia.org/wiki/GTFS](https://en.wikipedia.org/wiki/GTFS)

Un jeu GTFS statique est une **archive .zip** qui contient des **fichiers .txt** (CSV) inter-liés.

Les plus fréquents :

| Fichier              | Rôle                                                                   | Clés importantes         |
| -------------------- | ---------------------------------------------------------------------- | ------------------------ |
| `routes.txt`         | Lignes commerciales (bus, tram, métro…)                                | `route_id`, `route_type` |
| `trips.txt`          | Trajets (un “service” d’une route, dans un sens, un jour donné)        | `trip_id`, `route_id`    |
| `stops.txt`          | Arrêts (nom, coordonnées)                                              | `stop_id`, `stop_name`   |
| `stop_times.txt`     | Passage d’un trajet à un arrêt (ordre + horaires)                      | `trip_id`, `stop_id`     |
| `calendar.txt`       | Jours de service “réguliers” (lun-dim, dates de début/fin)             | `service_id`             |
| `calendar_dates.txt` | Exceptions au calendrier (jours ajoutés/retirés)                       | `service_id`, `date`     |
| `shapes.txt`         | Géométrie des trajets (polylignes) *(facultatif, utile pour la carte)* | `shape_id`               |

**Relations clés (simplifiées)**
`routes (route_id) ⟵ trips (route_id, trip_id) ⟵ stop_times (trip_id, stop_id) ⟶ stops (stop_id)`

**Particularités à connaître**

* Les horaires sont au format `HH:MM:SS` et peuvent **dépasser 24h** (ex. `25:12:00` pour un service après minuit).
* Les arrêts peuvent avoir des **relations parent/enfant** (`parent_station`) ; ici, on reste à l’échelle `stop_id`.
* `route_type` suit un code standard (0=tram, 1=métro, 2=train, 3=bus, 4=ferry…).
* Le GTFS **ne contient pas** de projection : les coordonnées des arrêts sont en **WGS84** (lat/lon).
* **Période de validité** : toujours vérifier les dates publiées avec la source.

## Sources

Ce fichier GTFS fait partie du jeu de données "Réseaux urbains de la Métropole Aix-Marseille-Provence". Il couvre les réseaux exploités par la RTM (Régie des Transports Métropolitains).

**Informations générales :**

* Réseau concerné : RTM
* Modes de transport couverts : bus, ferry, métro, tramway
* Nombre de lignes : 130
* Nombre de points d'arrêt : 2 677
* Nombre de zones d'arrêt : 0
* Données GTFS téléchargées depuis : [https://transport.data.gouv.fr/datasets/reseau-rtm-gtfs/](https://transport.data.gouv.fr/datasets/reseau-rtm-gtfs/)
* Format : archive `.zip` contenant des fichiers `.txt` structurés.
* **Période de validité des données : du 6 juin 2025 au 5 août 2025**

## Préparation & configuration

In [17]:
# Importation des bibliothèques necessaires
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from pathlib import Path

# Paramètres centralisés (déjà existants dans ton projet)
from config import OUTPUT_DIR, city, cadre_file

In [ ]:
base_path = Path("..") / "proxy" / "data" / "raw" /"transport" / "mamp-all.gtfs"

In [19]:
# Affichage chemin de fichiers
print("GTFS dir      :", base_path)
print("Sortie GPKG   :", OUTPUT_DIR)
print("Ville         :", city)
print("Cadre (fichier):", cadre_file)

GTFS dir      : ..\proxy\data\raw\transport\mamp-all.gtfs
Sortie GPKG   : ..\proxy\data\processed\overpass_results_Marseille
Ville         : Marseille
Cadre (fichier): ..\proxy\data\raw\insee\CadreMarseille.shp


## Lecture des fichiers GTFS + contrôles rapides

In [20]:
# Lecture 
stops      = pd.read_csv(base_path / "stops.txt")
routes     = pd.read_csv(base_path / "routes.txt")
trips      = pd.read_csv(base_path / "trips.txt")
stop_times = pd.read_csv(base_path / "stop_times.txt")

# Colonnes minimales 
required_cols = {
    "stops"     : {"stop_id", "stop_name", "stop_lat", "stop_lon"},
    "routes"    : {"route_id", "route_type"},
    "trips"     : {"trip_id", "route_id"},
    "stop_times": {"stop_id", "trip_id", "stop_sequence", "arrival_time", "departure_time"},
}
datasets = {"stops": stops, "routes": routes, "trips": trips, "stop_times": stop_times}

for name, df in datasets.items():
    missing = required_cols[name] - set(df.columns)
    print(f"{name:10s} - shape={df.shape} - missing={missing}")

C:\Users\hazim\AppData\Local\Temp\ipykernel_24040\1191352773.py:5: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  stop_times = pd.read_csv(base_path / "stop_times.txt")


stops      - shape=(16548, 15) - missing=set()
routes     - shape=(820, 13) - missing=set()
trips      - shape=(96160, 11) - missing=set()
stop_times - shape=(2092782, 10) - missing=set()


In [21]:
# Affichage de quelques lignes
display(stops.head(3))
display(routes.head(3))
display(trips.head(3))
display(stop_times.head(3))

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,wheelchair_boarding,import_id,city_name,ext_netex_id,postal_code
0,AST-AIX-21230-AIX-24090,NaN,08 mai 1945,NaN,43.522967,5.432991,NaN,NaN,1,NaN,0,NaN,Aix-en-Provence,NaN,NaN
1,MAMP-S30245,30245,08-mai-45,NaN,43.456311,5.558428,NaN,NaN,1,NaN,0,NaN,Fuveau,NaN,NaN
2,MAMP-S31127,31127,08-mai-45,NaN,43.458669,5.478744,NaN,NaN,1,NaN,0,NaN,Gardanne,NaN,NaN


,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_color,route_text_color,route_sort_order,direction0_name,direction1_name,route_group,import_id
0,ULY-1BIS,ULY,1BIS,Ligne 1Bis,NaN,3,6ABC87,FFFFFF,NaN,Istres,Miramas / Istres / Fos-sur-Mer,NaN,NaN
1,ENV-1,ENV,1,1 - Gare SNCF <> Barriol,1,3,FF0000,ffffff,NaN,Gare SNCF,Barriol,NaN,38
2,BDE-1,BDE,1,Avenue Général Leclerc - Gare Saint-Antoine,NaN,3,FFD800,000000,NaN,Marseille,Les Pennes-Mirabeau,NaN,NaN


,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed,import_id
0,ULY-1BIS,ULY-18663,ULY-1BISx1Ax3x1,Istres,Course 1,0,NaN,NaN,0,0,NaN
1,ULY-1BIS,ULY-18672,ULY-1BISx1Rx1x1,Istres,Course 1,0,NaN,NaN,0,0,NaN
2,ULY-1BIS,ULY-18663,ULY-1BISx5Rx2x1,Istres,Course 1,0,NaN,NaN,0,0,NaN


,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint
0,ULY-1BISx1Ax3x1,06:40:00,06:40:00,MAMP-22529,0,NaN,0,0,NaN,1
1,ULY-1BISx1Ax3x1,06:42:00,06:42:00,MAMP-20583,1,NaN,0,0,NaN,1
2,ULY-1BISx1Ax3x1,06:44:00,06:44:00,MAMP-25434,2,NaN,0,0,NaN,1


##  Vérifications (intégrité + horaires)

### Intégrité référentielle minimale

In [22]:
# Chaque trip_id de stop_times doit exister dans trips
unknown_trips = stop_times[~stop_times["trip_id"].isin(trips["trip_id"])].shape[0]
# Chaque route_id de trips doit exister dans routes
unknown_routes = trips[~trips["route_id"].isin(routes["route_id"])].shape[0]

print("trip_id inconnus dans stop_times:", unknown_trips)
print("route_id inconnus dans trips   :", unknown_routes)

trip_id inconnus dans stop_times: 0
route_id inconnus dans trips   : 0


Si >0, le jeu est inconsistant (ou incomplet) → consigner/traiter selon besoin.

### 3.2 Horaires > 24h (pédago)

In [23]:
def hhmmss_to_seconds(hms: str) -> int:
    """Transforme HH:MM:SS en secondes, supporte HH >= 24."""
    h, m, s = map(int, hms.split(":"))
    return h*3600 + m*60 + s

sample_times = stop_times["arrival_time"].dropna().head(5).tolist()
print("Exemples d'horaires:", sample_times)
print("En secondes        :", [hhmmss_to_seconds(t) for t in sample_times])

Exemples d'horaires: ['06:40:00', '06:42:00', '06:44:00', '06:47:00', '06:48:00']
En secondes        : [24000, 24120, 24240, 24420, 24480]


> *Pourquoi utile ?* Pour des analyses de fréquences/amplitudes, mieux vaut convertir en secondes (gère proprement le >24h).

## Chaînage logique — type majoritaire par arrêt

On détermine, pour chaque `stop_id`, le **mode qui passe le plus souvent** (bus/tram/métro).

In [25]:
# Liaison stop_times → trips → routes
stop_trip   = stop_times[["stop_id", "trip_id"]].drop_duplicates()
trip_route  = trips[["trip_id", "route_id"]]
route_type  = routes[["route_id", "route_type"]]

stops_type  = stop_trip.merge(trip_route, on="trip_id", how="left")
stops_type  = stops_type.merge(route_type, on="route_id", how="left")

In [26]:
# Type majoritaire par stop_id
mode_per_stop = (
    stops_type
    .groupby("stop_id")["route_type"]
    .agg(lambda s: s.value_counts(dropna=True).idxmax())
    .reset_index()
)

missing_rt = mode_per_stop["route_type"].isna().sum()
print("stop_id sans route_type (après jointures) :", missing_rt)
mode_per_stop.head()

stop_id sans route_type (après jointures) : 0


,stop_id,route_type
0,AIX-00026,3
1,AIX-10006,3
2,AIX-12744,3
3,AIX-12745,3
4,AIX-15109,3


## Typage clair (mapping route_type → libellé)

In [27]:
type_map = {0: "tram", 1: "metro", 3: "bus"}  # étends si besoin (ex: 4: ferry)

mode_per_stop = mode_per_stop[mode_per_stop["route_type"].isin(type_map.keys())].copy()
mode_per_stop["motive_id"] = mode_per_stop["route_type"].map(type_map)

mode_per_stop["motive_id"].value_counts()

motive_id
bus      10312
tram        80
metro       61
Name: count, dtype: int64

## Jointure avec `stops` + GeoDataFrame

In [28]:
stops_filtered = stops.merge(mode_per_stop[["stop_id", "motive_id"]], on="stop_id", how="inner")

gdf = gpd.GeoDataFrame(
    stops_filtered,
    geometry=gpd.points_from_xy(stops_filtered["stop_lon"], stops_filtered["stop_lat"]),
    crs="EPSG:4326"
).to_crs("EPSG:2154")

gdf = gdf[["stop_id", "stop_name", "motive_id", "geometry"]]
gdf.head()

,stop_id,stop_name,motive_id,geometry
0,AIX-21230,08 mai 1945,bus,POINT (896736.566 6272345.053)
1,AIX-24090,08 mai 1945,bus,POINT (896749.870 6272359.696)
2,MAMP-23431,08 mai 1945,bus,POINT (875842.544 6266900.430)
3,MAMP-25627,08 mai 1945,bus,POINT (875853.866 6266915.641)
4,BDE-23431,08-mai-45,bus,POINT (875879.396 6266917.678)


## Filtrage spatial par l’emprise (Marseille)

In [29]:
cadre = gpd.read_file(cadre_file).to_crs("EPSG:2154")

before = len(gdf)
gdf = gdf[gdf.geometry.within(cadre.unary_union)].copy()
after = len(gdf)

print(f"Arrêts avant filtre  : {before}")
print(f"Arrêts après filtre  : {after}")
print(f"Hors emprise supprimés: {before - after}")

Arrêts avant filtre  : 10453
Arrêts après filtre  : 5348
Hors emprise supprimés: 5105


**Interprétation** : si beaucoup de points sortent, vérifier CRS et exactitude du `cadre`.

## Petit diagnostic

In [30]:
print("Répartition par type (motive_id) :")
display(gdf["motive_id"].value_counts())

dup_after = gdf.duplicated(subset=["stop_id"]).sum()
print("Duplicats sur stop_id :", dup_after)

Répartition par type (motive_id) :


motive_id
bus      5207
tram       80
metro      61
Name: count, dtype: int64

Duplicats sur stop_id : 0


## Export GeoPackage (et formats annexes)

In [31]:
out_gpkg = Path(OUTPUT_DIR) / f"gtfs_stops_{city}.gpkg"
out_gpkg.parent.mkdir(parents=True, exist_ok=True)
gdf.to_file(out_gpkg, layer="arrets_tc", driver="GPKG")

print("Export GPKG :", out_gpkg, "| n =", len(gdf))

Export GPKG : ..\proxy\data\processed\overpass_results_Marseille\gtfs_stops_Marseille.gpkg | n = 5348


## Encapsulation réutilisable

*(fonction, prête pour le pipeline)*

In [32]:
def extract_gtfs_stops(export=False, output_gpkg=None):
    stops      = pd.read_csv(base_path / "stops.txt")
    routes     = pd.read_csv(base_path / "routes.txt")
    trips      = pd.read_csv(base_path / "trips.txt")
    stop_times = pd.read_csv(base_path / "stop_times.txt")

    stop_trip  = stop_times[["stop_id", "trip_id"]].drop_duplicates()
    trip_route = trips[["trip_id", "route_id"]]
    route_type = routes[["route_id", "route_type"]]

    stops_type = stop_trip.merge(trip_route, on="trip_id").merge(route_type, on="route_id")
    mode_per_stop = (
        stops_type.groupby("stop_id")["route_type"]
        .agg(lambda s: s.value_counts(dropna=True).idxmax())
        .reset_index()
    )

    type_map = {0: "tram", 1: "metro", 3: "bus"}
    mode_per_stop = mode_per_stop[mode_per_stop["route_type"].isin(type_map.keys())].copy()
    mode_per_stop["motive_id"] = mode_per_stop["route_type"].map(type_map)

    stops_filtered = stops.merge(mode_per_stop[["stop_id", "motive_id"]], on="stop_id", how="inner")
    gdf = gpd.GeoDataFrame(
        stops_filtered,
        geometry=gpd.points_from_xy(stops_filtered["stop_lon"], stops_filtered["stop_lat"]),
        crs="EPSG:4326"
    ).to_crs("EPSG:2154")

    cadre = gpd.read_file(cadre_file).to_crs("EPSG:2154")
    gdf = gdf[gdf.geometry.within(cadre.unary_union)].copy()
    gdf_final = gdf[["stop_id", "stop_name", "motive_id", "geometry"]]

    if export and output_gpkg:
        output_gpkg = Path(output_gpkg)
        output_gpkg.parent.mkdir(parents=True, exist_ok=True)
        gdf_final.to_file(output_gpkg, layer="arrets_tc", driver="GPKG")

    return gdf_final

In [ ]:
# Test
out_gpkg = Path(OUTPUT_DIR) / f"gtfs_stops_{city}.gpkg"
gdf_stops = extract_gtfs_stops(export=True, output_gpkg=out_gpkg)
print("Export OK :", out_gpkg.exists(), "| n =", len(gdf_stops))
display(gdf_stops.head())

C:\Users\hazim\AppData\Local\Temp\ipykernel_24040\416887354.py:5: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  stop_times = pd.read_csv(base_path / "stop_times.txt")


Export OK : True | n = 5348


,stop_id,stop_name,motive_id,geometry
15,BDE-20299,19 mars 62,bus,POINT (881349.247 6263961.020)
16,BDE-25735,19 mars 62,bus,POINT (881384.711 6263984.488)
18,RTM-00001532,3 Frères Barthélemy,bus,POINT (893705.902 6246701.204)
19,RTM-00001685,3 Ponts Canal,bus,POINT (897955.203 6244778.833)
20,RTM-00001687,3 Ponts Canal,bus,POINT (897976.470 6244787.497)


## Conclusion/ aller plus loin

* Le **GTFS** décrit une **offre théorique** (statique) — complémentaire, mais distinct, du **temps réel**.
* La **jointure logique** `stop_times → trips → routes` est le cœur de l’enrichissement.
* Les **horaires > 24h** sont courants : préférer une conversion en **secondes** pour les calculs.
* `route_type` peut inclure d’autres modes (ferry, train) : **adapte le mapping** à ton usage.
* Pour la suite : fréquences par arrêt (en combinant `stop_times` et `calendar/_dates`), amplitude horaire, maillage intermodal, etc.